In [3]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

# Export the model to TensorRT format
model.export(format="engine")  # creates 'yolov8n.engine'

# Load the exported TensorRT model
tensorrt_model = YOLO("yolov8n.engine")

# Run inference
results = tensorrt_model("https://ultralytics.com/images/bus.jpg")

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:00<00:00, 23.9MB/s]


WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.2.100 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla V100-PCIE-16GB, 16144MiB)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)
requirements: Ultralytics requirement ['onnxruntime-gpu'] not found, attempting AutoUpdate...


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 48.3 MB/s eta 0:00:0000:0100:01

requirements: AutoUpdate success ✅ 11.8s, installed 1 package: ['onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.2 opset 19...
ONNX: slimming with onnxslim 0.1.34...
ONNX: export success ✅ 13.3s, saved as 'yolov8n.onnx' (12.2 MB)

TensorRT: starting export with TensorRT 10.3.0...
[09/24/2024-12:38:48] [TRT] [I] [MemUsageChange] Init CUDA: CPU +2, GPU +0, now: CPU 569, GPU 11346 (MiB)
[09/24/2024-12:38:49] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +440, GPU +78, now: CPU 1165, GPU 11424 (MiB)
[09/24/2024-12:38:49] [TRT] [I] ----------------------------------------------------------------
[09/24/2024-12:38:49] [TRT] [I] Input filename:   yolov8n.onnx
[09/24/2024-12:38:49] [TRT] [I] ONNX IR version:  0.0.10
[09/24/2024-12:38:49] [TRT] [I] Opset version:    19
[09/24/2024-12:38:49] [TRT] [I] Produ

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 134k/134k [00:00<00:00, 1.60MB/s]


image 1/1 /env/kunal/tensor-rt/bus.jpg: 640x640 4 persons, 1 bus, 2.5ms
Speed: 7.8ms preprocess, 2.5ms inference, 154.5ms postprocess per image at shape (1, 3, 640, 640)


In [8]:
!apt install python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package python is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  2to3 python2-minimal python2 dh-python python-is-python3

E: Package 'python' has no installation candidate


In [11]:
!pip install onnx-simplifier

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.3 MB/s eta 0:00:00a 0:00:01


In [13]:
!python3 -m onnxsim yolov8n.onnx yolo_simplified.onnx

Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃            ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add        │ 8              │ 8                │
│ Concat     │ 19             │ 19               │
│ Constant   │ 144            │ 143              │
│ Conv       │ 64             │ 64               │
│ Div        │ 1              │ 1                │
│ MaxPool    │ 3              │ 3                │
│ Mul        │ 58             │ 58               │
│ Reshape    │ 5              │ 5                │
│ Resize     │ 2              │ 2                │
│ Sigmoid    │ 58             │ 58               │
│ Slice      │ 2              │ 2                │
│ Softmax    │ 1              │ 1                │
│ Split      │ 9              │ 9                │
│ Sub        │ 2              │ 2                │
│ Transpose  │ 1              │ 1                │
│ Model Size │ 12.2MiB        │ 12.

In [14]:
!trtexec --onnx=yolo_simplified.onnx --saveEngine=yolo.trt --fp16

&&&& RUNNING TensorRT.trtexec [TensorRT v100300] # trtexec --onnx=yolo_simplified.onnx --saveEngine=yolo.trt --fp16
[09/24/2024-12:52:09] [I] === Model Options ===
[09/24/2024-12:52:09] [I] Format: ONNX
[09/24/2024-12:52:09] [I] Model: yolo_simplified.onnx
[09/24/2024-12:52:09] [I] Output:
[09/24/2024-12:52:09] [I] === Build Options ===
[09/24/2024-12:52:09] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default, tacticSharedMem: default
[09/24/2024-12:52:09] [I] avgTiming: 8
[09/24/2024-12:52:09] [I] Precision: FP32+FP16
[09/24/2024-12:52:09] [I] LayerPrecisions: 
[09/24/2024-12:52:09] [I] Layer Device Types: 
[09/24/2024-12:52:09] [I] Calibration: 
[09/24/2024-12:52:09] [I] Refit: Disabled
[09/24/2024-12:52:09] [I] Strip weights: Disabled
[09/24/2024-12:52:09] [I] Version Compatible: Disabled
[09/24/2024-12:52:09] [I] ONNX Plugin InstanceNorm: Disabled
[09/24/2024-12:52:09] [I] TensorRT runtime: full
[09/24/2024-12:52:09] [I] Lean DLL Pa

In [15]:
!pwd

/env/kunal/tensor-rt
